In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing as pp


In [ ]:
df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_sim.csv", index_col=0)
df

In [ ]:
df2 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_dummy.csv", index_col=0)
df2

In [ ]:
df3 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_rank.csv", index_col=0)
df3

In [ ]:
df[df['rank'] == 5.0]

In [ ]:
def rank_from_col(x):
    if x == .5:
       return 1
    else:
        return 0 

In [ ]:
df['top'] = df['var'].apply(rank_from_col)


In [ ]:
df[df['var'] == .5]

In [ ]:
df

In [ ]:
test = df[['city_id', 'food_sim', 'attraction_sim', 'top']]
test

In [ ]:
test.info()

In [ ]:
test.set_index('city_id',append=True, inplace=True)


In [ ]:
test.top

In [ ]:
df2.reset_index(inplace=True)
df3.reset_index(inplace=True)

In [ ]:
idk = df2.drop(columns=['city'])
idk2 = df3.drop(columns=['city'])

In [ ]:
idk2

In [ ]:
idk.set_index(['user', 'label_id'], inplace=True)
idk2.set_index(['user', 'label_id'], inplace=True)

In [ ]:
idk['rank']

In [ ]:

idk.rename(columns={'rank': 'top'}, inplace=True)
idk2.rename(columns={'rank': 'top'}, inplace=True)

In [ ]:
idk

In [ ]:
blah = pd.DataFrame(scaled_features)

In [ ]:
idk2

In [ ]:
normalized = pp.normalize(idk2)
normalized = pd.DataFrame(normalized, index=idk2.index, columns=idk2.columns)
normalized

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(idk2.drop('top', axis=1),
                                                    idk2.top, test_size= 0.2, random_state=12)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(idk2.values)

scaled_features_df = pd.DataFrame(scaled_features, index=idk2.index, columns=idk2.columns)


In [ ]:
scaled_features_df

In [ ]:
x_train

### Logistic Regression

Accuary is .97
But precision and recall for 1 = 0

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', max_iter=10000).fit(x_train, y_train)
predictions_lr = clf.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_lr))

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(x_train, y_train)
predictions_kn = neigh.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_kn))

### SVM

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf')
svclassifier.fit(x_train, y_train)
predictions_svm = svclassifier.predict(x_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions_svm))
print(classification_report(y_test,predictions_svm))

In [ ]:
test.top.value_counts()

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)
predictions_dt = clf.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_dt))
print(confusion_matrix(y_test,predictions_dt))


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=12)
clf.fit(x_train, y_train)
predictions_rf = clf.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, predictions_rf))
print(confusion_matrix(y_test,predictions_rf))


In [ ]:
(y_test ==1).any()

In [ ]:
(y_test == 1).count()

In [ ]:
type(y_test)

In [ ]:
y_test.value_counts()

In [ ]:

idk[idk['top'] == 0]

In [ ]:
idk.top.value_counts()